<a href="https://colab.research.google.com/github/AschHarwood/social_unrest/blob/main/asch_harwood_state_predictions_neural_prophet_all_states_v4_12_10_21.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predicting Covid Social Unrest Over Time in the United States
Asch Harwood

asch@redhookmedialab.com

## Goal

Goal 1: Predict in realtime covid-related social unrest based on a changing "information environment" in the United States

Goal 2: Forecast up to five days in advance covid-related social unrest based on a changing "information environment" in the United States

## Dataset Time Range
- Feb 26, 2020 to May 24, 2021 
- 395 days

## Target

- Covid social unrest events as documented by the Armed Conflict Location and Event Dataset (ACLED)
- 5,462 events documented from Jan 13, 2020 to May 24, 2021
- Covid-related events extracted from ACLED using covid-relevant keyword search in the `notes` section
- By aggregating data to national level for model building, target represents daily mean protests nationally

## Features
- Roughly 436 features, which include timeseries and static data:

**Time Series**
  - Daily covid case and death metrics, New York Times, County-level
  - Daily aggregate mobility score derived from Facebook, Google, and Apple mobility data, County-level
  - Daily covid infodemic misinformation risk metrics, Covid19 Infodemics Observatory, created by Complex Multilayer Networks Lab, National
  - Covid-tagged Reddit comments and sentiment, State-level, (add publisher)
  - Daily covid-tagged tweets, 50-dimension embedding using a tensorflow google news pretrained model, County-level (add publisher)
  - Daily covid-relevant "local" news articles, 50-dimension embedding using a tensorflow google news pretrained model, geotagged to State-level based on state location of publisher. Articles urls and location extracted via GDELT and Mediacloud, text acquired via webscrapping
  - Weekly covid-related public policies, Kaiser Family Foundation, one hot encoded

**Static**
  - U.S. Census data, County-level
  - Governor and State-senate leader political affliation, Kaiser Family Foundation, State-level, one hot encoded


## Dataset shape
- Daily, county-level data: 1,472,412 rows, 437 columns
- Daily, use-level data: 

## Preprocessing and Modeling
Dataset is:
1. Passed to `Tsfresh` package for feature extraction (reality is this doesnt yield any real improvements in performance and is very computational expensive)

2. Aggregated to the daily national level and mean is calculated for all features and targets

2. Split into training (80%, 316 days) and test(20%, 79 days) set.

3. Scaled using `RobustScaler()`

4. PCA, with `n_components=0.9999`

5. Passed to Neural Prophet model

6. RMSE and MAE calculated for train and test data

7. Train and test predictions and actual are plotted

8. Residuals for 1 day lagged are plotted

## Analysis

- Tough problem to solve since covid-related unrest events are relatively rare, which yields a sparse/imbalanced dataset 
- Neverthless, while predictions on both the train and test sets are rather noisey, they do seem to capture at least some of the overall observable trends (i.e. protests going up, protests going down).
- With improvements to the dataset and additional experimentation, I think it might be possible to develop a model that performs relatively well in predicting at least a day out how changes in information influence future covid social unrest

# Install and Imports

In [ ]:
!pip install tsfresh

!pip install git+https://github.com/ourownstory/neural_prophet.git

!pip install wandb
!pip install us
import pandas as pd
pd.set_option('display.max_columns', 500)
import us
import plotly.express as px
from neuralprophet import NeuralProphet
from sklearn.preprocessing import RobustScaler
from sklearn.decomposition import PCA
from IPython.display import display, Markdown
import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import MinMaxScaler
import numpy as np
import tensorflow_hub as hub
import tensorflow as tf
from tsfresh.utilities.dataframe_functions import roll_time_series
from tsfresh import extract_features
from tsfresh import select_features
from tsfresh.utilities.dataframe_functions import impute
import wandb
wandb.login()

# Reading Data

In [5]:
#read in data 
df = pd.read_parquet('/content/drive/MyDrive/social_unrest_data/main_data_policy_text_embeddings_11_15_21.parquet')

#drop first few rows of data (because I am missing consecutive dates in the original dataset)
#this dataset is daily, county-level data consisting of roughly 1.5 million records
df = df.set_index('date')['2020-01-13':]

#create a lookup dictionary to fips codes to state names
state_lookup_dict = us.states.mapping('fips', 'name')

#reset index
df.reset_index(inplace=True)



In [6]:
df.head()

,date,state_fips,fips,cases_avg,cases_avg_per_100k,deaths_avg,deaths_avg_per_100k,cases,deaths,Mobility_score,info_tweets,info_tweets_T,info_tweets_RE,info_tweets_RT,info_tweets_local,info_tweets_verified,info_tweets_unverified,info_fact_reliable,info_fact_unreliable,info_fact_unknown,info_fact_unreliable_fraction,info_fact_shadow,info_fact_missing,info_fact_science,info_fact_msm,info_fact_satire,info_fact_clickbait,info_fact_political,info_fact_fake,info_fact_conspiracy,info_iri,info_iri_T,info_iri_RE,info_iri_RT,info_dyn_iri,info_dyn_iri_RE,info_dyn_iri_RT,reddit_comments_1,reddit_comments_2,reddit_sentiment,AREA_SQMI,E_TOTPOP,M_TOTPOP,E_HU,M_HU,E_HH,M_HH,E_POV,M_POV,E_UNEMP,M_UNEMP,E_PCI,M_PCI,E_NOHSDP,M_NOHSDP,E_AGE65,M_AGE65,E_AGE17,M_AGE17,E_DISABL,M_DISABL,E_SNGPNT,M_SNGPNT,E_MINRTY,M_MINRTY,E_LIMENG,M_LIMENG,E_MUNIT,M_MUNIT,E_MOBILE,M_MOBILE,E_CROWD,M_CROWD,E_NOVEH,M_NOVEH,E_GROUPQ,M_GROUPQ,EP_POV,MP_POV,EP_UNEMP,MP_UNEMP,EP_PCI,MP_PCI,EP_NOHSDP,MP_NOHSDP,EP_AGE65,MP_AGE65,EP_AGE17,MP_AGE17,EP_DISABL,MP_DISABL,EP_SNGPNT,MP_SNGPNT,EP_MINRTY,MP_MINRTY,EP_LIMENG,MP_LIMENG,EP_MUNIT,MP_MUNIT,EP_MOBILE,MP_MOBILE,EP_CROWD,MP_CROWD,EP_NOVEH,MP_NOVEH,EP_GROUPQ,MP_GROUPQ,EPL_POV,EPL_UNEMP,EPL_PCI,EPL_NOHSDP,SPL_THEME1,RPL_THEME1,EPL_AGE65,EPL_AGE17,EPL_DISABL,EPL_SNGPNT,SPL_THEME2,RPL_THEME2,EPL_MINRTY,EPL_LIMENG,SPL_THEME3,RPL_THEME3,EPL_MUNIT,EPL_MOBILE,EPL_CROWD,EPL_NOVEH,EPL_GROUPQ,SPL_THEME4,RPL_THEME4,SPL_THEMES,RPL_THEMES,F_POV,F_UNEMP,F_PCI,F_NOHSDP,F_THEME1,F_AGE65,F_AGE17,F_DISABL,F_SNGPNT,F_THEME2,F_MINRTY,F_LIMENG,F_THEME3,F_MUNIT,F_MOBILE,F_CROWD,F_NOVEH,F_GROUPQ,F_THEME4,F_TOTAL,E_UNINSUR,M_UNINSUR,EP_UNINSUR,MP_UNINSUR,E_DAYPOP,Rate_filled,protest_class,year,month,day,Governor Political Affiliation_Democrat,Governor Political Affiliation_Republican,Governor Political Affiliation_Unknown,State Senate Majority Political Affiliation_Democrat,State Senate Majority Political Affiliation_Other,State Senate Majority Political Affiliation_Republican,State Senate Majority Political Affiliation_Unknown,State House Majority Political Affiliation_Democrat,State House Majority Political Affiliation_Republican,State House Majority Political Affiliation_Unknown,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,protest_count,State Is Easing Social Distancing Measures_Yes,State Is Easing Social Distancing Measures_unknown,Stay at Home Order_-,Stay at Home Order_Counties of High Transmission (New),Stay at Home Order_During Curfew Hours,Stay at Home Order_High-Risk Groups,Stay at Home Order_Lifted,Stay at Home Order_New Stay at Home Order,Stay at Home Order_Regions of High Transmission,Stay at Home Order_Rolled Back to High Risk Groups,Stay at Home Order_Statewide,Stay at Home Order_Statewide (New),Stay at Home Order_unknown,Mandatory Quarantine for Travelers_-,Mandatory Quarantine for Travelers_All Air Travelers,Mandatory Quarantine for Travelers_All Travelers,Mandatory Quarantine for Travelers_All Travelers (New),Mandatory Quarantine for Travelers_From Certain States,Mandatory Quarantine for Travelers_From Certain States (New),Mandatory Quarantine for Travelers_International Travelers,Mandatory Quarantine for Travelers_Lifted,Mandatory Quarantine for Travelers_Rolled Back to Certain States,Mandatory Quarantine for Travelers_Rolled Back to International Travel,Mandatory Quarantine for Travelers_unknown,Non-Essential Business Closures_-,Non-Essential Business Closures_All Non-Essential Businesses Open,Non-Essential Business Closures_All Non-Essential Businesses Open with Limits,Non-Essential Business 

In [7]:
df.shape

(1472412, 437)

In [8]:
df.protest_count.sum()

5462.0

In [9]:
df.date

0         2020-01-13
1         2020-01-13
2         2020-01-13
3         2020-01-13
4         2020-01-13
             ...    
1472407   2021-05-24
1472408   2021-05-24
1472409   2021-05-24
1472410   2021-05-24
1472411   2021-05-24
Name: date, Length: 1472412, dtype: datetime64[ns]

In [10]:
df.info(max_cols=500)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1472412 entries, 0 to 1472411
Data columns (total 437 columns):
 #   Column                                                                                                          Non-Null Count    Dtype         
---  ------                                                                                                          --------------    -----         
 0   date                                                                                                            1472412 non-null  datetime64[ns]
 1   state_fips                                                                                                      1472412 non-null  object        
 2   fips                                                                                                            1472412 non-null  int64         
 3   cases_avg                                                                                                       1472412 non-null  f

In [11]:
#drop columns that cannot be aggregated
df = df.drop(['fips', 'year', 'month', 'day'], axis=1)

#set date as index
df.set_index('date', inplace=True)

#calculate the mean value for data from all counties
df = df.resample('D').mean()

#drops a few duplicated columns (which come from the census data)
df = df.loc[:, ~df.T.duplicated()]

'''
Protest count, the target we are trying to predict, is currently displayed as the daily county average, 
meaning the number of daily protest counts for each county is aggregated and divided by the total number of counties.
The new number reflects the average daily protests across the county. Given that in the 498 days of data for 3,200 counties, 
there were roughly 5000 total covid-related public actions, the mean number is in the thousandths, and calculated to 6 decimal places,
I rounded to just three decimals and multiple by 1000 to assit in comparison.
Given that these numbers are not the actual number of protests, we can intepret these numbers based on there pattern over time -- are
covid-related social actions increasing or decreasing?


'''
df.protest_count = df.protest_count.round(3)*1000

In [12]:
df

,cases_avg,cases_avg_per_100k,deaths_avg,deaths_avg_per_100k,cases,deaths,Mobility_score,info_tweets,info_tweets_T,info_tweets_RE,info_tweets_RT,info_tweets_local,info_tweets_verified,info_tweets_unverified,info_fact_reliable,info_fact_unreliable,info_fact_unknown,info_fact_unreliable_fraction,info_fact_shadow,info_fact_missing,info_fact_science,info_fact_msm,info_fact_satire,info_fact_clickbait,info_fact_political,info_fact_fake,info_fact_conspiracy,info_iri,info_iri_T,info_iri_RE,info_iri_RT,info_dyn_iri,info_dyn_iri_RE,info_dyn_iri_RT,reddit_comments_1,reddit_comments_2,reddit_sentiment,AREA_SQMI,E_TOTPOP,M_TOTPOP,E_HU,M_HU,E_HH,M_HH,E_POV,M_POV,E_UNEMP,M_UNEMP,E_PCI,M_PCI,E_NOHSDP,M_NOHSDP,E_AGE65,M_AGE65,E_AGE17,M_AGE17,E_DISABL,M_DISABL,E_SNGPNT,M_SNGPNT,E_MINRTY,M_MINRTY,E_LIMENG,M_LIMENG,E_MUNIT,M_MUNIT,E_MOBILE,M_MOBILE,E_CROWD,M_CROWD,E_NOVEH,M_NOVEH,E_GROUPQ,M_GROUPQ,EP_POV,MP_POV,EP_UNEMP,MP_UNEMP,EP_NOHSDP,MP_NOHSDP,EP_AGE65,MP_AGE65,EP_AGE17,MP_AGE17,EP_DISABL,MP_DISABL,EP_SNGPNT,MP_SNGPNT,EP_MINRTY,MP_MINRTY,EP_LIMENG,MP_LIMENG,EP_MUNIT,MP_MUNIT,EP_MOBILE,MP_MOBILE,EP_CROWD,MP_CROWD,EP_NOVEH,MP_NOVEH,EP_GROUPQ,MP_GROUPQ,EPL_POV,EPL_UNEMP,EPL_PCI,EPL_NOHSDP,SPL_THEME1,RPL_THEME1,EPL_AGE65,EPL_AGE17,EPL_DISABL,EPL_SNGPNT,SPL_THEME2,RPL_THEME2,EPL_MINRTY,EPL_LIMENG,SPL_THEME3,RPL_THEME3,EPL_MUNIT,EPL_MOBILE,EPL_CROWD,EPL_NOVEH,EPL_GROUPQ,SPL_THEME4,RPL_THEME4,SPL_THEMES,RPL_THEMES,F_POV,F_UNEMP,F_PCI,F_NOHSDP,F_THEME1,F_AGE65,F_AGE17,F_DISABL,F_SNGPNT,F_THEME2,F_MINRTY,F_LIMENG,F_THEME3,F_MUNIT,F_MOBILE,F_CROWD,F_NOVEH,F_GROUPQ,F_THEME4,F_TOTAL,E_UNINSUR,M_UNINSUR,EP_UNINSUR,MP_UNINSUR,E_DAYPOP,Rate_filled,protest_class,Governor Political Affiliation_Democrat,Governor Political Affiliation_Republican,Governor Political Affiliation_Unknown,State Senate Majority Political Affiliation_Democrat,State Senate Majority Political Affiliation_Other,State Senate Majority Political Affiliation_Republican,State House Majority Political Affiliation_Democrat,State House Majority Political Affiliation_Republican,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,protest_count,State Is Easing Social Distancing Measures_Yes,State Is Easing Social Distancing Measures_unknown,Stay at Home Order_-,Stay at Home Order_Counties of High Transmission (New),Stay at Home Order_During Curfew Hours,Stay at Home Order_High-Risk Groups,Stay at Home Order_Lifted,Stay at Home Order_New Stay at Home Order,Stay at Home Order_Regions of High Transmission,Stay at Home Order_Rolled Back to High Risk Groups,Stay at Home Order_Statewide,Stay at Home Order_Statewide (New),Mandatory Quarantine for Travelers_-,Mandatory Quarantine for Travelers_All Air Travelers,Mandatory Quarantine for Travelers_All Travelers,Mandatory Quarantine for Travelers_All Travelers (New),Mandatory Quarantine for Travelers_From Certain States,Mandatory Quarantine for Travelers_From Certain States (New),Mandatory Quarantine for Travelers_International Travelers,Mandatory Quarantine for Travelers_Lifted,Mandatory Quarantine for Travelers_Rolled Back to Certain States,Mandatory Quarantine for Travelers_Rolled Back to International Travel,Non-Essential Business Closures_-,Non-Essential Business Closures_All Non-Essential Businesses Open,Non-Essential Business Closures_All Non-Essential Businesses Open with Limits,Non-Essential Business Closures_All Non-Essential Businesses Permitted to Reopen,Non-Essential Business Closures_All Non-Essential Businesses Permitted to Reopen,Non-Essential Business Closures_All Non-Essential Businesses Permitted to Reopen wit

In [13]:

run = wandb.init()
artifact = run.use_artifact('rhml/all_state_covid_unrest/state_embeddings:v0', type='dataset')
artifact_dir = artifact.download()
articles = pd.read_parquet(artifact_dir)

wandb: Currently logged in as: rhml (use `wandb login --relogin` to force relogin)


In [14]:
articles.reset_index(inplace=True)
grouped_states = articles.groupby([pd.Grouper(key='publish_date', freq='D'), 'state']).mean()


In [15]:
articles = grouped_states.reset_index().drop('state', axis=1).set_index('publish_date').resample('D').mean()

In [16]:
df_articles_embedding = df.merge(articles, how='inner', left_index=True, right_index=True)

In [17]:
df_articles_embedding.reset_index(inplace=True)
df_articles_embedding

,index,cases_avg,cases_avg_per_100k,deaths_avg,deaths_avg_per_100k,cases,deaths,Mobility_score,info_tweets,info_tweets_T,info_tweets_RE,info_tweets_RT,info_tweets_local,info_tweets_verified,info_tweets_unverified,info_fact_reliable,info_fact_unreliable,info_fact_unknown,info_fact_unreliable_fraction,info_fact_shadow,info_fact_missing,info_fact_science,info_fact_msm,info_fact_satire,info_fact_clickbait,info_fact_political,info_fact_fake,info_fact_conspiracy,info_iri,info_iri_T,info_iri_RE,info_iri_RT,info_dyn_iri,info_dyn_iri_RE,info_dyn_iri_RT,reddit_comments_1,reddit_comments_2,reddit_sentiment,AREA_SQMI,E_TOTPOP,M_TOTPOP,E_HU,M_HU,E_HH,M_HH,E_POV,M_POV,E_UNEMP,M_UNEMP,E_PCI,M_PCI,E_NOHSDP,M_NOHSDP,E_AGE65,M_AGE65,E_AGE17,M_AGE17,E_DISABL,M_DISABL,E_SNGPNT,M_SNGPNT,E_MINRTY,M_MINRTY,E_LIMENG,M_LIMENG,E_MUNIT,M_MUNIT,E_MOBILE,M_MOBILE,E_CROWD,M_CROWD,E_NOVEH,M_NOVEH,E_GROUPQ,M_GROUPQ,EP_POV,MP_POV,EP_UNEMP,MP_UNEMP,EP_NOHSDP,MP_NOHSDP,EP_AGE65,MP_AGE65,EP_AGE17,MP_AGE17,EP_DISABL,MP_DISABL,EP_SNGPNT,MP_SNGPNT,EP_MINRTY,MP_MINRTY,EP_LIMENG,MP_LIMENG,EP_MUNIT,MP_MUNIT,EP_MOBILE,MP_MOBILE,EP_CROWD,MP_CROWD,EP_NOVEH,MP_NOVEH,EP_GROUPQ,MP_GROUPQ,EPL_POV,EPL_UNEMP,EPL_PCI,EPL_NOHSDP,SPL_THEME1,RPL_THEME1,EPL_AGE65,EPL_AGE17,EPL_DISABL,EPL_SNGPNT,SPL_THEME2,RPL_THEME2,EPL_MINRTY,EPL_LIMENG,SPL_THEME3,RPL_THEME3,EPL_MUNIT,EPL_MOBILE,EPL_CROWD,EPL_NOVEH,EPL_GROUPQ,SPL_THEME4,RPL_THEME4,SPL_THEMES,RPL_THEMES,F_POV,F_UNEMP,F_PCI,F_NOHSDP,F_THEME1,F_AGE65,F_AGE17,F_DISABL,F_SNGPNT,F_THEME2,F_MINRTY,F_LIMENG,F_THEME3,F_MUNIT,F_MOBILE,F_CROWD,F_NOVEH,F_GROUPQ,F_THEME4,F_TOTAL,E_UNINSUR,M_UNINSUR,EP_UNINSUR,MP_UNINSUR,E_DAYPOP,Rate_filled,protest_class,Governor Political Affiliation_Democrat,Governor Political Affiliation_Republican,Governor Political Affiliation_Unknown,State Senate Majority Political Affiliation_Democrat,State Senate Majority Political Affiliation_Other,State Senate Majority Political Affiliation_Republican,State House Majority Political Affiliation_Democrat,State House Majority Political Affiliation_Republican,0_x,1_x,2_x,3_x,4_x,5_x,6_x,7_x,8_x,9_x,10_x,11_x,12_x,13_x,14_x,15_x,16_x,17_x,18_x,19_x,20_x,21_x,22_x,23_x,24_x,25_x,26_x,27_x,28_x,29_x,30_x,31_x,32_x,33_x,34_x,35_x,36_x,37_x,38_x,39_x,40_x,41_x,42_x,43_x,44_x,45_x,46_x,47_x,48_x,49_x,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,protest_count,State Is Easing Social Distancing Measures_Yes,State Is Easing Social Distancing Measures_unknown,Stay at Home Order_-,Stay at Home Order_Counties of High Transmission (New),Stay at Home Order_During Curfew Hours,Stay at Home Order_High-Risk Groups,Stay at Home Order_Lifted,Stay at Home Order_New Stay at Home Order,Stay at Home Order_Regions of High Transmission,Stay at Home Order_Rolled Back to High Risk Groups,Stay at Home Order_Statewide,Stay at Home Order_Statewide (New),Mandatory Quarantine for Travelers_-,Mandatory Quarantine for Travelers_All Air Travelers,Mandatory Quarantine for Travelers_All Travelers,Mandatory Quarantine for Travelers_All Travelers (New),Mandatory Quarantine for Travelers_From Certain States,Mandatory Quarantine for Travelers_From Certain States (New),Mandatory Quarantine for Travelers_International Travelers,Mandatory Quarantine for Travelers_Lifted,Mandatory Quarantine for Travelers_Rolled Back to Certain States,Mandatory Quarantine for Travelers_Rolled Back to International Travel,Non-Essential Business Closures_-,Non-Essential Business Closures_All Non-Essential Businesses Open,Non-Essential Business Closures_All Non-Essential Businesses Open with Limits,Non-Essential Business Closures_All Non-Essential Businesses Permitted to Reopen,Non-Essential Business Closures_All Non-Essential Businesse

In [18]:
df_articles_embedding.protest_count.sum()

1704.0

# NP with article embeddings

In [19]:


run = wandb.init()
artifact = run.use_artifact('rhml/all_state_covid_unrest/ts_fresh_timeseries_filtered_features:v1', type='dataset')
artifact_dir = artifact.download()

wandb: Downloading large artifact ts_fresh_timeseries_filtered_features:v1, 173.08MB. 1 files... Done. 0:0:0


In [20]:
df_rolled= pd.read_parquet(artifact_dir)

In [21]:
df_complete = pd.concat([df_articles_embedding[['index', 'protest_count']], df_rolled], axis=1)

In [22]:
df_complete.rename(columns={'index': 'ds', 'protest_count': 'y'}, inplace=True)

In [23]:
#defines the config file for logging model parameters and results on Weights and Biases (wandb.com)
config = dict (
    rolling_value = 5,
    n_forecasts = 5,
    n_lags = 15,
    num_hidden_layers = 3,
    n_changepoints=175,
    trend_reg=0.1,
    batch_size=16,
    ar_sparsity=None,
    changepoints_range=1,
    valid_p=0.2,
    n_components=1,
    learning_rate=0.05,
    growth='linear',
    epochs=200
)

wandb.init(project='all_state_covid_unrest',
           notes='developing model for all states aggregated',
           config=config)

config = wandb.config

In [24]:
#creates a copy of the dataset
state = df_complete.copy()



In [25]:




print('')

#define neural prophet model
#uses 15 days lag to predict up to five days in the future
m = NeuralProphet(n_forecasts=wandb.config.n_forecasts, n_lags=config.n_lags, num_hidden_layers=config.num_hidden_layers,  n_changepoints=config.num_hidden_layers, trend_reg=config.trend_reg, 
                  batch_size=config.batch_size, ar_sparsity=config.ar_sparsity, 
                  changepoints_range=config.changepoints_range, 
                  yearly_seasonality=False, weekly_seasonality=False, daily_seasonality=False, growth=config.growth, learning_rate=config.learning_rate, epochs=config.epochs)

#test trian split
df_train, df_test = m.split_df(df=state, freq='D', valid_p=config.valid_p)
df_train.set_index(['ds', 'y'], inplace=True)
df_test.set_index(['ds', 'y'], inplace=True)

#scale data, robust scaler to deal with outliers
scaler = RobustScaler()
scaler.fit(df_train)
train_scaled = scaler.transform(df_train)
test_scaled = scaler.transform(df_test)

#dimensionality reduction
pca = PCA(n_components=.9999)
pca.fit(train_scaled)
train_scaled = pca.transform(train_scaled)
test_scaled = pca.transform(test_scaled)
train= pd.concat([df_train.reset_index()[['ds', 'y']], pd.DataFrame(train_scaled)], axis=1)
test= pd.concat([df_test.reset_index()[['ds', 'y']], pd.DataFrame(test_scaled)], axis=1)
train.columns = train.columns.astype(str)
test.columns = test.columns.astype(str)

#add regressors to model
lagged_regressors = list(train.loc[:, '0':].columns)
lagged_regressors
m.add_lagged_regressor(lagged_regressors)

#fit model, generate predictions
metrics_train = m.fit(df=train, freq='D')
metrics_test = m.test(df=test)
train_forecast = m.predict(train)
test_forecast =m.predict(test)

display(Markdown('###Train Metrics'))
display(metrics_train.describe())
print('')

display(Markdown('###Test Metrics'))
display(metrics_test.describe())


#Plotting and Logging Data
train_forecast.rename(columns={'ds': 'Date', 'y': 'Actual', 'yhat1': 'Prediction_1', 'yhat2': 'Prediction_2', 'yhat3':'Prediction_3', 'yhat4': 'Prediction_4', 'yhat5': 'Prediction_5'}, inplace=True)
train_forecast[['Prediction_1', 'Prediction_2', 'Prediction_3', 'Prediction_4', 'Prediction_5']] = train_forecast[['Prediction_1', 'Prediction_2', 'Prediction_3', 'Prediction_4', 'Prediction_5']].astype(float).apply(lambda x: np.round(x, 2))                                                                                                          
test_forecast.rename(columns={'ds': 'Date', 'y': 'Actual', 'yhat1': 'Prediction_1', 'yhat2': 'Prediction_2', 'yhat3':'Prediction_3', 'yhat4': 'Prediction_4', 'yhat5': 'Prediction_5'}, inplace=True)
test_forecast[['Prediction_1', 'Prediction_2', 'Prediction_3', 'Prediction_4', 'Prediction_5']] = test_forecast[['Prediction_1', 'Prediction_2', 'Prediction_3', 'Prediction_4', 'Prediction_5']].astype(float).apply(lambda x: np.round(x, 2))                                                                                                          

train_melted = pd.melt(train_forecast[['Date', 'Actual', 'Prediction_1', 'Prediction_2', 'Prediction_3', 'Prediction_4', 'Prediction_5']], id_vars='Date', value_vars=['Actual', 'Prediction_1', 'Prediction_2', 'Prediction_3', 'Prediction_4', 'Prediction_5'])

test_melted = pd.melt(test_forecast[['Date', 'Actual', 'Prediction_1', 'Prediction_2', 'Prediction_3', 'Prediction_4', 'Prediction_5']], id_vars='Date', value_vars=['Actual', 'Prediction_1', 'Prediction_2', 'Prediction_3', 'Prediction_4', 'Prediction_5'])
train_melted.rename(columns={'value': '5_Day_Rolling_Avg_Protests', 'variable': 'Prediction_Type'}, inplace=True)

test_melted.rename(columns={'value': '5_Day_Rolling_Avg_Protests', 'variable': 'Prediction_Type'}, inplace=True)
fig = px.line(train_melted, x='Date', y='5_Day_Rolling_Avg_Protests', color='Prediction_Type', title=f'Train Actual and Predictions: All US')
fig.show()


fig = px.line(train_forecast, x='Date', y='residual1', title='Train Residuals')
fig.show()


fig = px.line(test_melted, x='Date', y='5_Day_Rolling_Avg_Protests', color='Prediction_Type', title=f'Test Actual and Predictions: ALL US')
fig.show()




fig = px.line(test_forecast, x='Date', y='residual1', title='Test Residuals')
fig.show()




INFO - (NP.config.__post_init__) - Note: Trend changepoint regularization is experimental.


Epoch[200/200]: 100%|██████████| 200/200 [00:13<00:00, 14.67it/s, SmoothL1Loss=0.0241, MAE=1.47, RMSE=2.26, RegLoss=0.000166]
INFO - (NP.forecaster._evaluate) - Validation metrics:    SmoothL1Loss    MAE   RMSE
1         0.039  2.028  2.796


###Train Metrics

,SmoothL1Loss,MAE,RMSE,RegLoss
count,200.000000,200.000000,200.000000,2.000000e+02
mean,453.631747,4537.933701,20221.022677,4.000022e-05
std,2097.914904,20979.680266,93452.241811,5.846068e-05
min,0.024058,1.467173,2.229671,0.000000e+00
25%,0.028472,1.625213,2.453348,0.000000e+00
50%,0.039601,1.918895,2.966916,3.018393e-09
75%,0.043270,2.044072,3.139777,7.691034e-05
max,14610.387361,146108.851268,624642.161893,1.662534e-04


###Test Metrics

,SmoothL1Loss,MAE,RMSE
count,1.000000,1.000000,1.000000
mean,0.038574,2.028493,2.795834
std,NaN,NaN,NaN
min,0.038574,2.028493,2.795834
25%,0.038574,2.028493,2.795834
50%,0.038574,2.028493,2.795834
75%,0.038574,2.028493,2.795834
max,0.038574,2.028493,2.795834
